In [3]:
import torch

# Humandoid MARL
from Humanoid_MARL import envs
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs
from Humanoid_MARL.utils.utils import load_reward_config
from Humanoid_MARL import CONFIG_REWARD
import mediapy as media

In [4]:
config = {
        'num_timesteps': 100_000_000,
        'eval_frequency': 10,
        'episode_length': 1000,
        'unroll_length': 10,
        'num_minibatches': 32,
        'num_update_epochs': 8,
        'discounting': 0.97,
        'learning_rate': 3e-4,
        'entropy_cost': 1e-3,
        'num_envs': 2048,
        'batch_size': 512,
        'env_name': "humanoids",
        'render' : True,
        'device' : 'cpu',
        'model_path' : "../models/20240226_082641_ppo_humanoids_standing.pt",
        'video_length' : 300,
        'device_idx': 1
    }


env_config = load_reward_config(CONFIG_REWARD, config['env_name'])
env = envs.create(
        config['env_name'],
        auto_reset=False,
        **env_config,
    )

env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['device'], get_jax_state=True)

# env warmup
observation = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['device'])
env.step(action)
agents = load_models(config['model_path'], Agent, device=config['device'])
jax_states = []
num_steps = 1000

eval_reward = 0.0
episodes = torch.zeros((), device = config['device'])
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    logits, action = get_agent_actions(agents, observation, env.obs_dims)
    jax_state, observation, reward, done, info = env.step(Agent.dist_postprocess(action[0]))
    episodes += torch.sum(done)
    jax_states.append(jax_state)
    print(f"{i} | {info} | DONE [{done}] | Reward [{reward}]")
    eval_reward += reward
print(f"Total Reward | {eval_reward / episodes}")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

In [ ]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 